In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from ncrp import Topic, Doc, init, sample, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_ncrp, get_docs
from configure import get_config

# load config & data 

In [3]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [4]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

In [5]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(9006, 1995)

# run

## initialize log

In [6]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [7]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=0, config=config)
train_docs = get_docs(instances_train, config)
dev_docs = get_docs(instances_dev, config)
test_docs = get_docs(instances_test, config)
init(train_docs, dev_docs, test_docs, topic_root)

0 0 0 

## run

In [8]:
time_start = time.time()
while epoch < config.n_epochs:
    sample(train_docs, dev_docs, test_docs, topic_root)
    ppl_train = get_perplexity(train_docs, topic_root)
    ppl_dev = get_perplexity(dev_docs, topic_root)
    
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    time_log = int(time.time() - time_start)
    time_start = time.time()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, '%.0f'%ppl_dev, '%.0f'%ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2], '%.2f'%depth_spec[3], \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series    
    display(log_df)
    get_freq_tokens_ncrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN: VALID: TEST: SPEC:             HIER:      
   Time  Ep Ct    PPL    PPL   PPL     1     2     3 CHILD OTHER
0   669   0  0   1037   1348  1095  0.02  0.39  0.43  0.71  0.68
1   766   1  0   1031   1339  1089  0.02  0.42  0.44  0.78  0.77
2   773   2  0   1022   1329  1082  0.02  0.42  0.45  0.80  0.79
3   769   3  0   1012   1321  1072  0.02  0.41  0.45  0.78  0.78
4   773   4  0   1002   1313  1063  0.02  0.41  0.45  0.77  0.76
5   841   5  0    995   1307  1056  0.02  0.42  0.45  0.79  0.79
6   850   6  0    990   1303  1051  0.02  0.42  0.45  0.79  0.78
7   855   7  0    985   1298  1047  0.02  0.42  0.45  0.78  0.78
8   854   8  0    980   1294  1042  0.02  0.42  0.45  0.78  0.78
9   853   9  0    975   1290  1038  0.03  0.42  0.45  0.78  0.77
10  852  10  0    971   1287  1035  0.03  0.42  0.45  0.77  0.77
11  818  11  0    967   1285  1032  0.03  0.42  0.45  0.77  0.77
12  854  12  0    964   1283  1029  0.03  0.42  0.45  0.77  0.77
13  839  13  0    961   1280  1026  0.03  0.41  0.45  0.73  0.73
14  774  14  0    959   1275  1024  0.03  0.42  0.45  0.77  0.76
15  798  15  0    956   1272  1021  0.03  0.42  0.45  0.76  0.76
16  860  16  0    953   1269  1018  0.03  0.42  0.45  0.76  0.76
17  851  17  0    951   1267  1015  0.03  0.42  0.45  0.76  0.76
18  884  18  0    948   1264  1013  0.03  0.42  0.45  0.76  0.76
19  765  19  0    946   1263  1010  0.03  0.42  0.45  0.76  0.76
20  855  20  0    943   1262  1008  0.03  0.42  0.45  0.75  0.75
21  816  21  0    941   1259  1006  0.03  0.42  0.45  0.75  0.75
22  804  22  0    939   1256  1004  0.04  0.42  0.45  0.75  0.75
23  829  23  0    937   1253  1002  0.04  0.42  0.45  0.75  0.74
24  889  24  0    934   1250  1000  0.04  0.42  0.45  0.75  0.74
25  806  25  0    932   1247   997  0.04  0.42  0.45  0.74  0.74
26  813  26  0    930   1246   995  0.04  0.42  0.45  0.74  0.74
27  837  27  0    927   1242   991  0.04  0.42  0.45  0.73  0.73
28  815  28  0    924   1238   988  0.04  0.43  0.45  0.73  0.73
29  773  29  0    921   1233   984  0.04  0.42  0.45  0.73  0.72
30  894  30  0    917   1230   979  0.04  0.41  0.45  0.69  0.68
31  840  31  0    912   1224   976  0.04  0.41  0.45  0.72  0.72
32  756  32  0    907   1216   971  0.04  0.40  0.45  0.67  0.67
33  791  33  0    902   1206   966  0.04  0.40  0.45  0.67  0.67
34  840  34  0    899   1201   963  0.04  0.40  0.45  0.67  0.67
35  773  35  0    896   1197   960  0.04  0.40  0.45  0.67  0.67
36  759  36  0    894   1194   958  0.04  0.40  0.45  0.67  0.67
37  680  37  0    891   1191   955  0.04  0.40  0.45  0.67  0.67
38  684  38  0    889   1188   952  0.05  0.40  0.45  0.67  0.67
39  700  39  0    886   1184   950  0.05  0.40  0.45  0.67  0.67
40  602  40  0    884   1182   947  0.05  0.40  0.45  0.67  0.67
41  637  41  0    881   1178   945  0.05  0.40  0.45  0.67  0.66
42  708  42  0    878   1176   943  0.05  0.40  0.45  0.67  0.66
43  660  43  0    875   1174   939  0.05  0.41  0.45  0.71  0.71
44  660  44  0    873   1170   937  0.05  0.41  0.45  0.71  0.70
45  401  45  0    870   1166   934  0.05  0.41  0.45  0.71  0.70
46  400  46  0    868   1162   931  0.05  0.41  0.45  0.71  0.70
47  457  47  0    865   1160   928  0.06  0.41  0.45  0.71  0.70
48  425  48  0    864   1157   926  0.06  0.41  0.45  0.71  0.70
49  370  49  0    862   1153   923  0.06  0.41  0.45  0.71  0.70

 0 9006 351518.0 use write get one article know like file work problem
   0-1 3847 160479.0 write one key article use say people get know make
     0-1-1 271 22.0 hear function remember apparently father tend mother module joke policy
     0-1-2 3576 3824.0 islam islamic war muslim study morality resurrection church country rights
   0-2 1337 84832.0 game team year play go get write player think win
     0-2-1 1337 111.0 edge jewish finger brown ok warning also rock kid higher
   0-3 11 1400.0 appear copy art new vs cover annual rider app issue
     0-3-2 11 0.0 bhj series ram dave knowledge prevent james throw washington legal
   0-4 613 48160.0 space launch use nasa satellite system year information research program
     0-4-1 538 138.0 blood whole level medical lower human percent tend affect measure
     0-4-2 75 23.0 signature pgp begin organization media btw sit message club database
   0-5 3170 40784.0 drive use disk one entry ground wire power new controller
     0-5-1 2724 519